<h1 align = 'center'>Guessing Games</h1>
<h3 align = 'center'>machine learning, one step at a time</h3>
<h3 align = 'center'>Step 7. A Random Walk Through a Maze</h3>

#### 7. A random walk through a maze

Imagine a maze that's 4x4:

In [ ]:
from maze import Maze
maze = Maze()          # make a new maze...
print(maze)            # ...and print it

Let's take a walk through the maze.

We always start in the upper left corner (that's why it says (1) on the maze).

We can move N,S,E,W by calling the __step()__ function:

In [ ]:
maze.step('E') # take one step to the East
print(maze)

Let's walk through the entire maze by taking the correct sequence of steps...

In [ ]:
maze.step('E') # (3) take another step to the East
maze.step('S') # (4) and then one step to the South
maze.step('E') # then (5) East, (6) South, (7) East
maze.step('S')
maze.step('S')
print(maze)

<hr>
__That was easy, because we know a thing or two about mazes.__

In fact, we know a lot about mazes. It may be true that we know everything that there is to know about mazes.

Before we could find our way, we needed to be able to answer all of these questions:
- what is a maze?<br>
- is this problem a maze, or some other kind of problem?<br>
- what is the difference between an open space and a blocked space?<br>
- what does it mean to move north, south, east, or west?<br>
- what does it mean to stay in bounds?<br>
- where is the exit?<br>
- what condition represents successfully completing the maze?<br>

Turns out we are _total maze experts_. We have perfect knowledge of the maze, so we can take the optimum path right away. Nobody knows more about mazes than we do.
<hr>
__But here's a question for a total maze expert: is any of that knowledge is really necessary?__

What if we turned the problem on its head, and asked: _what is the absolute minimum amount of information needed to traverse a maze?_

Do we really need to know about boundaries, and blocked spaces, or the idea of an entrance or exit?

What if all we could possibly know included:
- where we are (that's our __state__, a unique description of our circumstances).
- what __actions__ we can take (not what each __action__ means, just what choices are available).
- the __result__ of taking a given __action__ when in a given __state__:
    - a __reward__, for find the exit.
    - a __penalty__, for going out of bounds or onto a blocked space.
    - or __nothing__ for moving to an open space.

If our maze was set up to give just that limited amount of feedback, it could be solved, without knowing anything about the nature of a maze. The maze can be solved solely by examining _state transitions_. So can lots of games or puzzles, or other sorts of problems, all with the same types of algorithms.

It turns out our maze provides exactly the feedback we need! (not because mazes do that; really, because this is a computer science lesson).

Let's take a look:

In [ ]:
state = maze.reset() # store the initial state returned by the maze
print('initial state is =', state)

That makes sense. Our initial coordinates are (0,0).

What about knowing what actions are available, or taking an action?

The maze provides a function called __action_space()__, which returns all the available actions, and __sample()__, which returns a random action from among the available actions:

In [ ]:
# what actions are available?
print('Here are all the possible actions:', maze.action_space())

for n in range(5):
    print('...a random action might be =', maze.sample())

And when we were using __step()__, we ignored the feedback: for each __step()__, the maze returns three values:
- an updated __state__.
- a __reward__ (+1), a __penalty__ (-1), or __nothing__ (0).
- a __done__ flag, which is the equivalent of 'game over'.

It looks like this:

In [ ]:
maze.reset()                                              # start over
print('I moved east and this happened:', maze.step('E'))  # move to an open space, no reward or penalty
print('I moved east and this happened:', maze.step('E'))  # and another open space...
print('I moved east and this happened:', maze.step('E'))  # and now a blocked space, penalty! game over!

So time for our first algorithm: take a random walk through the maze.

Here is code that takes one random walk, showing each step along the way. Run it several times. How far can you go?

In [ ]:
# take random walks through a 4x4 maze until one attempt succeeds in reaching the exit
maze.reset()
done = False
while not done:
    observation, reward, done = maze.step(maze.sample())  # sample() provides a random action (N,S,E,W)
    print(maze,observation,reward,done)

<hr>
***Excercises***<p>

Complete the program to take random walks through the maze, until the exit is reached.
- How many attempts does that take?
- If you succeed 1,000 times, how many attempts are required, on average, for each success?

In [ ]:
from maze import Maze

# take random walks through a 4x4 maze until you succeed in reaching the exit
maze = Maze()
attempts = 0
successful = False 

while not successful:
    attempts += 1
    
    #################################
    #                               #
    #  YOUR CODE GOES HERE...       #
    #                               #
    #  try a random walk... if      #
    #  you dont' find the exit,     #
    #  try again. And again.        #
    #                               #
    #################################
    
print(attempts)    